In [1]:
!ls data/nak/자동/

도개_2013.xlsx	도개_2019.xlsx	신암_2017.xlsx	풍양_2015.xlsx	회상_2016.xlsx
도개_2014.xlsx	도개_2020.xlsx	신암_2018.xlsx	풍양_2016.xlsx	회상_2017.xlsx
도개_2015.xlsx	신암_2013.xlsx	신암_2019.xlsx	풍양_2017.xlsx	회상_2018.xlsx
도개_2016.xlsx	신암_2014.xlsx	신암_2020.xlsx	풍양_2018.xlsx	회상_2019.xlsx
도개_2017.xlsx	신암_2015.xlsx	풍양_2013.xlsx	풍양_2019.xlsx	회상_2020.xlsx
도개_2018.xlsx	신암_2016.xlsx	풍양_2014.xlsx	풍양_2020.xlsx


In [2]:
!pwd

/home/jaewon/work/water-quality


In [3]:
%matplotlib widget
import pandas as pd
import numpy as np
from glob import glob
import os
import datetime
import matplotlib.pyplot as plt
    
from tensorflow.keras.layers import Input, Concatenate, Dot, Add, ReLU, Activation
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from tensorflow import keras
import datetime

In [4]:
# for Korean in plot
import matplotlib
import matplotlib.font_manager as fm
fm.get_fontconfig_fonts()
font_location = '/usr/share/fonts/truetype/nanum/NanumGothicCoding.ttf'
#font_location = '/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc'
# font_location = 'C:/Windows/Fonts/NanumGothic.ttf' # For Windows
fprop = fm.FontProperties(fname=font_location)

In [5]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

In [181]:

#folder = 'data/han/자동'
#folder = 'data/nak/자동'
#folder = 'data/geum/자동'
folder = 'data/yeong/자동'

# ## 한강



# ## 자동
#file_names = [
#         ["가평_2013.xlsx", "가평_2014.xlsx", "가평_2015.xlsx", "가평_2016.xlsx", "가평_2017.xlsx", "가평_2018.xlsx", "가평_2019.xlsx", "가평_2020.xlsx"],
#         ["서상_2013.xlsx", "서상_2014.xlsx", "서상_2015.xlsx", "서상_2016.xlsx", "서상_2017.xlsx", "서상_2018.xlsx", "서상_2019.xlsx", "서상_2020.xlsx"],
#         ["의암호_2013.xlsx", "의암호_2014.xlsx", "의암호_2015.xlsx", "의암호_2016.xlsx", "의암호_2017.xlsx", "의암호_2018.xlsx", "의암호_2019.xlsx","의암호_2020.xlsx"]
#             ]


#file_names = [
#        ["도개_2013.xlsx", "도개_2014.xlsx", "도개_2015.xlsx", "도개_2016.xlsx", "도개_2017.xlsx", "도개_2018.xlsx", "도개_2019.xlsx", "도개_2020.xlsx"],
#        ["신암_2013.xlsx", "신암_2014.xlsx", "신암_2015.xlsx", "신암_2016.xlsx", "신암_2017.xlsx", "신암_2018.xlsx", "신암_2019.xlsx", "신암_2020.xlsx"],
#        ["풍양_2013.xlsx", "풍양_2014.xlsx", "풍양_2015.xlsx", "풍양_2016.xlsx", "풍양_2017.xlsx", "풍양_2018.xlsx", "풍양_2019.xlsx", "풍양_2020.xlsx"]
#             ]


#file_names = [
#        ["현도_2013.xlsx", "현도_2014.xlsx", "현도_2015.xlsx", "현도_2016.xlsx", "현도_2017.xlsx", "현도_2018.xlsx", "현도_2019.xlsx", "현도_2020.xlsx"],
#        ["대청호_2013.xlsx", "대청호_2014.xlsx", "대청호_2015.xlsx", "대청호_2016.xlsx", "대청호_2017.xlsx", "대청호_2018.xlsx", "대청호_2019.xlsx", "대청호_2020.xlsx"]
#             ]
        
file_names = [
        ["나주_2013.xlsx", "나주_2014.xlsx", "나주_2015.xlsx", "나주_2016.xlsx", "나주_2017.xlsx", "나주_2018.xlsx", "나주_2019.xlsx", "나주_2020.xlsx"],
        ["용봉_2013.xlsx", "용봉_2014.xlsx", "용봉_2015.xlsx", "용봉_2016.xlsx", "용봉_2017.xlsx", "용봉_2018.xlsx", "용봉_2019.xlsx", "용봉_2020.xlsx"],
        ["우치_2013.xlsx", "우치_2014.xlsx", "우치_2015.xlsx", "우치_2016.xlsx", "우치_2017.xlsx", "우치_2018.xlsx", "우치_2019.xlsx", "우치_2020.xlsx"]
             ]



## 수질
# file_names = [['가평천3_2016.xlsx','가평천3_2017.xlsx','가평천3_2018.xlsx', '가평천3_2019.xlsx'],
#               ['대성리_2016.xlsx','대성리_2017.xlsx','대성리_2018.xlsx', '대성리_2019.xlsx'],
#               ['청평댐3_2016.xlsx','청평댐3_2017.xlsx','청평댐3_2018.xlsx', '청평댐3_2019.xlsx'],
#               ['춘천댐1_2016.xlsx','춘천댐1_2017.xlsx','춘천댐1_2018.xlsx', '춘천댐1_2019.xlsx'],
#               ['대성리_2016.xlsx','대성리_2017.xlsx','대성리_2018.xlsx', '대성리_2019.xlsx'],
#               ['의암_2016.xlsx','의암_2017.xlsx','의암_2018.xlsx', '의암_2019.xlsx'],
#               ['춘성교_2016.xlsx','춘성교_2017.xlsx','춘성교_2018.xlsx', '춘성교_2019.xlsx'],
#               ['남이섬_2016.xlsx','남이섬_2017.xlsx','남이섬_2018.xlsx', '남이섬_2019.xlsx'],
#               ['청평댐3_2016.xlsx','청평댐3_2017.xlsx','청평댐3_2018.xlsx', '청평댐3_2019.xlsx'],
#               ['춘천_2016.xlsx','춘천_2017.xlsx','춘천_2018.xlsx', '춘천_2019.xlsx']
#              ]


## AWS
# file_names = [['남이섬_2016.xlsx','남이섬_2017.xlsx','남이섬_2018.xlsx', '남이섬_2018.xlsx'],
#               ['청평_2016.xlsx', '청평_2017.xlsx', '청평_2018.xlsx', '청평_2019.xlsx'],
#               ['화천_2016.xlsx', '화천_2017.xlsx', '화천_2018.xlsx', '화천_2019.xlsx']]
 

    
## 낙동강
# #자동
# file_names = [['도개_2016.xlsx','도개_2017.xlsx','도개_2018.xlsx', '도개_2019.xlsx'],
#               ['신암_2016.xlsx','신암_2017.xlsx','신암_2018.xlsx', '신암_2019.xlsx'],
#               ['회상_2016.xlsx','회상_2017.xlsx','회상_2018.xlsx', '회상_2019.xlsx']]


## 금강
# #자동
# file_names = [['현도_2016.xlsx','현도_2017.xlsx','현도_2018.xlsx', '현도_2019.xlsx'],
#               ['대청호_2016.xlsx','대청호_2017.xlsx','대청호_2018.xlsx', '대청호_2019.xlsx'],
#               ['옥천천_2016.xlsx','옥천천_2017.xlsx','옥천천_2018.xlsx', '옥천천_2019.xlsx'],
#               ['장계_2016.xlsx','장계_2017.xlsx','장계_2018.xlsx', '장계_2019.xlsx']]
             

    
    
## 영산강
#자동
# file_names = [['나주_2016.xlsx','나주_2017.xlsx','나주_2018.xlsx', '나주_2019.xlsx'],
#               ['지석천_2016.xlsx','지석천_2017.xlsx','지석천_2018.xlsx', '지석천_2019.xlsx'],
#               ['용봉_2016.xlsx','용봉_2017.xlsx','용봉_2018.xlsx', '용봉_2019.xlsx'],
#               ['서창교_2016.xlsx','서창교_2017.xlsx','서창교_2018.xlsx', '서창교_2019.xlsx'],
#               ['우치_2016.xlsx','우치_2017.xlsx','우치_2018.xlsx', '우치_2019.xlsx']]
              

In [182]:
target_col = '용존산소'
# 각각의 강 마다, 측정소 별로 column을 맞춰야함
# 0: 자동, 1: 수질, 2: AWS(ASOS), 
han = [':,[26,27,28,29,30,31,32,33]', ':,28:45', ':,26:29']

#[26,27,28,29,30,31,32,33]

nakdong = [':,2:', ':,2:12']
gum = [':,2:'] #, ':,28:45', ':,26:29']
# gain_han = ['save/han/1', 'save/han/2', 'save/han/3']

# interpolation_option = True
interpolation_option = False, False
# gain_save_path = gain_han[2]

iloc_val = han[0]
iloc_val

':,[26,27,28,29,30,31,32,33]'

In [183]:
def make_columns(df):
    #column_list = ['측정날짜', '측정소명', '수온', '수소이온농도','전기전도도', '용존산소', '총유기탄소', '총질소', '총인', '클로로필-a']
    column_list = ['ymdh', 'stn_name', 'tmpr_value', 'ph_value','do_value', 'ec_value', 'toc_value', '총질소_값', '총인_값', '클로로필-a_값']
    list_df = pd.DataFrame(columns=column_list)
    list_df
    df = df.drop(columns=df.columns.difference(column_list))
    new_column = list_df.columns.difference(df.columns)
#     print(new_column)
    if not new_column.empty :
        print("Make_columns")
        for i in range(new_column.shape[0]):
            df[new_column[i]] = pd.Series()
#     print('columns')
#     print(df.columns)
    return df


In [184]:
def make_timeseries(df, interpolate=None):
    print(df.shape)
    date_col = df.columns[0]
    df[date_col] = pd.to_datetime(df[date_col])
    df = df[df[date_col].notna()]

    year = pd.DatetimeIndex(df[date_col]).year.astype(np.int64)

    start = str(year[0]) + "-01-01 00:00"
#     start

    end = str(year[0]) + "-12-31 23:00"
#     end

    time_series = pd.date_range(start = start, end = end, freq='H')
    time_series
    time_series = pd.DataFrame(time_series)
    time_series.columns = [date_col]   
    time_series

    df = pd.concat([time_series, df], axis=0)
    df = df.drop_duplicates([date_col], keep="last")
    df = df.sort_values([date_col], axis=0)
    
    if interpolate :
        print('interpolation')
        df = df.interpolate(metod='spline', order=2, limit_direction='both')
        
    print(df.shape)
    return df

In [185]:
def make_dataframe(file_names, iloc_val, interpolate=None):
    day = 24*60*60
    year = (365.2425)*day

    df_full = []
    df = []
    
    for loc in range(len(file_names)):
    
        df_loc = []
        for y in range(len(file_names[loc])):
            path = os.path.join(folder, file_names[loc][y])
            print(file_names[loc][y])
            df_tmp = make_timeseries(pd.read_excel(path), interpolate=interpolate)
#             df_tmp = pd.read_excel(path).dropna(axis='columns', how = 'all')
            df_tmp = make_columns(df_tmp)
            
    #         df_tmp = pd.read_excel(path).dropna(axis='columns', how = 'all')
    #         print(df_tmp.head) 
            df_loc.append(df_tmp)
#             df_loc.append(pd.read_excel(path))
        df_full.append(pd.concat(df_loc))
    #     df.append(df_full[loc].iloc[:,2:11])
        df.append(eval(f"df_full[loc].iloc[{iloc_val}]"))
    #     df.append(df_full[loc].iloc[:, [2,3,4,5,6,7,10]])
    #     df.append(df_full[loc].iloc[:, 2:11])
        date_time = pd.to_datetime(df_full[loc].iloc[:, 0], format='%Y.%m.%d %H:%M', utc=True)
        
        print(type(date_time))
        
        
        timestamp_s = date_time.map(datetime.datetime.timestamp)
        df[loc]['Day sin'] = np.sin(timestamp_s * (2 * np.pi / day))
        df[loc]['Day cos'] = np.cos(timestamp_s * (2 * np.pi / day))
        df[loc]['Year sin'] = np.sin(timestamp_s * (2 * np.pi / year))
        df[loc]['Year cos'] = np.cos(timestamp_s * (2 * np.pi / year))
        df[loc] = df[loc].reset_index(drop=True)
        
#         print(df[loc].shape[0])
#         rows = df[loc].shape[0]
#         percentage = round(rows * 0.7)
#         df[loc] = df[loc].dropna(axis='columns',  thresh=percentage)
        

    
    return df

In [186]:
def make_timeseries_by_day(df, interpolation=None):

    if interpolation[1] == False:
        return df

    time_day = []
    year = int(str(df.iloc[0, 0])[0:4])

    if df.shape[0] == 1:
        print('row in')
        ori_time = str(df.iloc[0, 0]) + "-06-15 12:00"
        ori_time = datetime.datetime.strptime(ori_time, '%Y-%m-%d %H:%M')
        df.iloc[0, 0] = ori_time
        return df
    elif df.shape[0] == 2:
        if df.iloc[0, 0] == df.iloc[1, 0]:
            ori_time = str(df.iloc[0, 0]) + "-03-15 12:00"
            ori_time = datetime.datetime.strptime(ori_time, '%Y-%m-%d %H:%M')
            df.iloc[0, 0] = ori_time

            ori_time = str(df.iloc[1, 0]) + "-09-15 12:00"
            ori_time = datetime.datetime.strptime(ori_time, '%Y-%m-%d %H:%M')
            df.iloc[1, 0] = ori_time
            return df


    div_cnt = df[df.columns[0]].value_counts().sort_index().tolist()

    for i in range(len(div_cnt)):

        month = calendar.monthrange(year, i + 1)
        for j in range(div_cnt[i]):
            time_val = datetime.timedelta(days=(month[1] / div_cnt[i] * j) + 1)

            time_day.append(
                '-' + str(time_val.days) + ' ' + str(time.strftime('%H:00:00', time.gmtime(time_val.seconds))))

    df.iloc[:, 0] = df.iloc[:, 0] + time_day

    return df


def make_timeseries(df, interpolation=None, iloc_val= None, loc=0, first_file_no=0, month=12, day=31):

    date_col = df.columns[0]
    df[date_col] = pd.to_datetime(df[date_col])
    df = df[df[date_col].notna()]
    df = df.dropna(thresh=3)

    year = pd.DatetimeIndex(df[date_col]).year.astype(np.int64)
    if loc==0 and first_file_no==0:
        month_tmp = pd.DatetimeIndex(df[date_col]).month.astype(np.int64)
        day_tmp = pd.DatetimeIndex(df[date_col]).day.astype(np.int64)
        month = month_tmp[-1] 
        day = day_tmp[-1] 
    else: 
        month = month
        day = day

    start = str(year[0]) + "-01-01 00:00"    #     start
    end = str(year[-1]) + "-" +str(month) + "-" + str(day) + " 23:00"#     end

    time_series = pd.date_range(start=start, end=end, freq='H')

    time_series = pd.DataFrame(time_series)
    time_series.columns = [date_col]

    time_series = pd.concat([time_series, df], axis=0)
    time_series = time_series.drop_duplicates([date_col], keep="last")
    time_series = time_series.sort_values([date_col], axis=0)


    if interpolation[0]:
        for i in range(1, df.shape[1], 1):
            idx = df.iloc[:, i].dropna()

            if idx.shape[0] != 0:
                time_series.iloc[0, i] = idx.iloc[0]
                time_series.iloc[-1, i] = idx.iloc[-1]

    return time_series, month, day


def make_dataframe(directory_path, file_names, iloc_val, interpolation=None, first_file_no=0, month=12, day=31):
    day_for_sincos = 24 * 60 * 60
    year_for_sincos = (365.2425) * day_for_sincos

    df_full = []
    df = []


    for loc in range(len(file_names)):

        df_loc = []
        for y in range(len(file_names[loc])):
            path = os.path.join(directory_path, file_names[loc][y])

            df_tmp = pd.read_excel(path)

            df_tmp = make_timeseries_by_day(df = df_tmp, interpolation=interpolation)

            df_loc.append(df_tmp)


        df_loc = pd.concat(df_loc)

        df_loc, month, day = make_timeseries(df_loc, interpolation=interpolation, iloc_val=iloc_val, month=month, day=day, loc=loc, first_file_no=first_file_no)


        df_full.append(df_loc)


        inter = eval(f"df_full[loc].iloc[{iloc_val}]")

        df.append(inter)

        date_time = pd.to_datetime(df_full[loc].iloc[:, 0], format='%Y.%m.%d %H:%M', utc=True)
        timestamp_s = date_time.map(datetime.datetime.timestamp)

        df[loc].insert(df[loc].shape[1], 'Day sin', np.sin(timestamp_s * (2 * np.pi / day_for_sincos)))
        df[loc].insert(df[loc].shape[1], 'Day cos', np.cos(timestamp_s * (2 * np.pi / day_for_sincos)))
        df[loc].insert(df[loc].shape[1], 'Year sin', np.sin(timestamp_s * (2 * np.pi / year_for_sincos)))
        df[loc].insert(df[loc].shape[1], 'Year cos', np.cos(timestamp_s * (2 * np.pi / year_for_sincos)))

        df[loc] = df[loc].reset_index(drop=True)

        if interpolation[0]:
            df[loc] = df[loc].interpolate(method='polynomial', order=3, limit_direction='both')



    return df, date_time.reset_index(drop=True), month, day

In [187]:
def normalize(df):
    # normalize data
    df_all = pd.concat(df)
    
    train_mean = df_all.mean()
    train_std = df_all.std()
    for i in range(len(file_names)):
        df[i] = (df[i]-train_mean)/train_std
    
    
    return df_all, train_mean, train_std

In [188]:
def min_max_normalize(df):
    # normalize data
    df_all = pd.concat(df)
    
    train_min = df_all.min()
    train_max = df_all.max()
    for i in range(len(file_names)):
        df[i] = (df[i]-train_min)/(train_max - train_min)       
    
    
    return df_all, train_min, train_max

In [189]:
##for normal
# df = make_dataframe(file_names, iloc_val)
## for interpolation
df, _, _, _ = make_dataframe(folder, file_names, iloc_val, interpolation=interpolation_option)
df_all, train_mean, train_std = normalize(df)
# df_all, train_mean, train_std = min_max_normalize(df)

In [190]:
train_mean, train_std

(tmpr_value    1.753058e+01
 ph_value      7.248738e+00
 do_value      8.294042e+00
 ec_value      3.042814e+02
 toc_value     4.088865e+00
 총질소_값         3.796938e+00
 총인_값          9.639838e-02
 클로로필-a_값      3.783906e+01
 Day sin       2.145782e-15
 Day cos      -3.012806e-15
 Year sin      4.063537e-02
 Year cos      9.321348e-03
 dtype: float64,
 tmpr_value     8.289807
 ph_value       0.666076
 do_value       3.735530
 ec_value      85.444680
 toc_value      1.061826
 총질소_값          1.885537
 총인_값           0.050966
 클로로필-a_값      37.730903
 Day sin        0.707109
 Day cos        0.707109
 Year sin       0.709027
 Year cos       0.703951
 dtype: float64)

## 

In [191]:
# just for Create WidnowGenerator 
train_df = df[0]
val_df = df[0]
test_df = df[0]

In [192]:
class WindowGenerator():
  def __init__(self, input_width, label_width, shift,
               train_df=train_df, val_df=val_df, test_df=test_df,
            #train_df=None, val_df=None, test_df=None,
#                out_features = None,
               out_features = None, out_num_features = None,
               label_columns=None):
    # Store the raw data.
    self.train_df = train_df
    self.val_df = val_df
    self.test_df = test_df

    # Work out the label column indices.
    self.label_columns = label_columns
    if label_columns is not None:
      self.label_columns_indices = {name: i for i, name in
                                    enumerate(label_columns)}
    self.column_indices = {name: i for i, name in
                           enumerate(train_df.columns)}

    # Work out the window parameters.
    self.input_width = input_width
    self.label_width = label_width
    self.shift = shift

    self.total_window_size = input_width + shift

    self.input_slice = slice(0, input_width)
    self.input_indices = np.arange(self.total_window_size)[self.input_slice]

    self.label_start = self.total_window_size - self.label_width
    self.labels_slice = slice(self.label_start, None)
    self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

  def __repr__(self):
    return '\n'.join([
        f'Total window size: {self.total_window_size}',
        f'Input indices: {self.input_indices}',
        f'Label indices: {self.label_indices}',
        f'Label column name(s): {self.label_columns}'])

In [193]:
def split_window(self, features):
  inputs = features[:, self.input_slice, :]
  labels = features[:, self.labels_slice, :]
  if self.label_columns is not None:
    labels = tf.stack(
        [labels[:, :, self.column_indices[name]] for name in self.label_columns],
        axis=-1)

  # Slicing doesn't preserve static shape information, so set the shapes
  # manually. This way the `tf.data.Datasets` are easier to inspect.
  inputs.set_shape([None, self.input_width, None])
  labels.set_shape([None, self.label_width, None])

  return inputs, labels

WindowGenerator.split_window = split_window

In [194]:
def plot(self, model=None, plot_col='T (degC)', max_subplots=3):
  inputs, labels = self.example
  plt.figure(figsize=(10, 8))
  plot_col_index = self.column_indices[plot_col]
  max_n = min(max_subplots, len(inputs))
  for n in range(max_n):
    plt.subplot(3, 1, n+1)
    plt.ylabel(f'{plot_col} [normed]', fontproperties=fprop)
    plt.plot(self.input_indices, inputs[n, :, plot_col_index],
             label='Inputs', marker='.', zorder=-10)

    if self.label_columns:
      label_col_index = self.label_columns_indices.get(plot_col, None)
    else:
      label_col_index = plot_col_index

    if label_col_index is None:
      continue

    plt.scatter(self.label_indices, labels[n, :, label_col_index],
                edgecolors='k', label='Labels', c='#2ca02c', s=64)
    if model is not None:
      predictions = model(inputs)
      plt.scatter(self.label_indices, predictions[n, :, label_col_index],
                  marker='X', edgecolors='k', label='Predictions',
                  c='#ff7f0e', s=64)

    if n == 0:
      plt.legend()

  plt.xlabel('Time [h]')

WindowGenerator.plot = plot

In [195]:
@property
def train(self):
  return self.make_dataset(self.train_df)

@property
def val(self):
  return self.make_dataset(self.val_df)

@property
def test(self):
  return self.make_dataset(self.test_df)

@property
def example(self):
  """Get and cache an example batch of `inputs, labels` for plotting."""
  result = getattr(self, '_example', None)
  if result is None:
    # No example batch was found, so get one from the `.train` dataset
    result = next(iter(self.train))
    # And cache it for next time
    self._example = result
  return result
@property
def example2(self):
  """Get and cache an example batch of `inputs, labels` for plotting."""
  result = getattr(self, '_example2', None)
  if result is None:
    # No example batch was found, so get one from the `.train` dataset
    result = next(iter(self.val))
    # And cache it for next time
    self._example = result
  return result
@property
def example3(self):
  """Get and cache an example batch of `inputs, labels` for plotting."""
  result = getattr(self, '_example3', None)
  if result is None:
    # No example batch was found, so get one from the `.train` dataset
    result = next(iter(self.test))
    # And cache it for next time
    self._example = result
  return result


WindowGenerator.train = train
WindowGenerator.val = val
WindowGenerator.test = test
WindowGenerator.example = example
WindowGenerator.example2 = example2
WindowGenerator.example3 = example3

In [196]:
def sample_batch_index(total, batch_size):
    '''Sample index of the mini-batch.

    Args:
        - total: total number of samples
        - batch_size: batch size

    Returns:
        - batch_idx: batch index
    '''
    total_idx = np.random.permutation(total)
    batch_idx = total_idx[:batch_size]
    return batch_idx

In [197]:
def binary_sampler(p, shape):
  '''Sample binary random variables.
  
  Args:
    - p: probability of 1
    - shape: matrix shape
    
  Returns:
    - binary_random_matrix: generated binary random matrix.
  '''
  unif_random_matrix = np.random.uniform(0., 1., size = shape)
  binary_random_matrix = 1*(unif_random_matrix < p)
  return binary_random_matrix

In [198]:
def uniform_sampler(low, high, shape):
  '''Sample uniform random variables.
  
  Args:
    - low: low limit
    - high: high limit
    - rows: the number of rows
    - cols: the number of columns
    
  Returns:
    - uniform_random_matrix: generated uniform random matrix.
  '''
  return np.random.uniform(low, high, size = shape)

In [199]:
class MissData(object):
    def __init__(self, load_dir=None):
        if load_dir:
            self.missarr = np.load(os.path.join(load_dir, 'miss.npy'))
            self.idxarr = np.load(os.path.join(load_dir, 'idx.npy'))
            
    def make_missdata(self, data_x, missrate=0.2):
        data = data_x.copy()
        rows, cols = data_x.shape
        total_no = rows*cols
        total_miss_no = np.round(total_no*missrate).astype(int)
        total_idx = self.idxarr.shape[0]
        idxarr = self.idxarr
        missarr = self.missarr
        #print(total_miss_no)
        miss_no = 0
        cum_no = self.idxarr[:,3:4]
        cum_no = cum_no.reshape((total_idx))
        cum_sum = np.max(cum_no)
        #print(cum_no)
        #print(total_idx)
        while True:
            loc_count = np.around(np.random.random()*cum_sum)
            #print('loc_count =', loc_count)
            idx = len(cum_no[cum_no <= loc_count])-1
            #print(cum_no[cum_no <= loc_count])
            #print('idx =', idx)
            startnan = idxarr[idx][0]
            nanlen = idxarr[idx][2]
            loc = np.around(np.random.random()*(rows-nanlen)).astype(int)
            #print('loc =', loc)
            #print(loc_count, idx)
            #print(idxarr[idx])
            #data_copy = data[loc:loc+nanlen].copy()
            data_copy = data[loc:loc+nanlen]
            #print('startnan=', startnan)
            #isnan = missarr[startnan:startnan+nanlen].copy()
            isnan = missarr[startnan:startnan+nanlen]
            #print('isnan =',isnan)
            miss_no += idxarr[idx][1]
            if (miss_no > total_miss_no):
                break
            data_copy[isnan==1] = np.nan
            data[loc:loc+nanlen] = data_copy
        #print('miss_data =', data)
        return data
    
    def save(data, max_tseq, save_dir='save'):
        no, dim = data.shape
        #print((no, dim))
        isnan = np.isnan(data).astype(int)
        isany = np.any(isnan, axis=1).astype(int)
        shifted = np.roll(isany, 1)
        shifted[0] = 1
        #print(isnan)
        #print(isany.astype(int))
        #print(shifted)
        startnan = ((isany == 1) & (shifted ==0)).astype(int)
        #print(startnan)
        group = startnan.cumsum()
        group = group*isany
#         print(group)
        n = np.max(group)
#         print(n)
        missarr = None
        cum_no = 0
        rowidx = 0
        for i in range(1, n+1):
            g = (group == i).astype(int)
            i = np.argmax(g)
            rows = g.sum()
            #print(len)
            #print(i)
            #print(type(missarr))
            if rows <= max_tseq:
                nanseq = isnan[i:i+rows, :]
                no = np.sum(nanseq)
                #print(no)
                if missarr is None:
                    missarr = nanseq
                    idxarr = np.array([[rowidx, no, rows, cum_no]])
                else:
                    missarr = np.concatenate((missarr, nanseq))
                    idxarr = np.concatenate((idxarr, [[rowidx, no, rows, cum_no]]), axis=0)
                cum_no += no
                rowidx += rows

        print(idxarr)
        miss_npy_file = os.path.join(save_dir, 'miss.npy')
        idx_npy_file = os.path.join(save_dir, 'idx.npy')
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)
        np.save(miss_npy_file, missarr)
        np.save(idx_npy_file, idxarr)
        print('miss_data file saved')

In [200]:
def interpolate(np_data, max_gap=3):
    #n = np_data.shape[1]
    data = pd.DataFrame(np_data)
    #data[0][0] = np.nan
    #data[0][1] = np.nan
    #data[0][2] = np.nan
    #data[data.columns[0]][0] = np.nan
    #data[data.columns[0]][1] = np.nan
    #data[data.columns[0]][2] = np.nan
    
    # create mask
    mask = data.copy()
    grp = ((mask.notnull() != mask.shift().notnull()).cumsum())
    grp['ones'] = 1
    for i in data.columns:
        mask[i] = (grp.groupby(i)['ones'].transform('count') < max_gap) | data[i].notnull()
    data = data.interpolate(method='polynomial', order=5, limit=max_gap, axis=0).bfill()[mask]
    return data.to_numpy()
    #return data
    
#filled_data = interpolate(norm_data, max_gap=3)
#np.arange(0, 5, dtype=int)
#['%d'%val for val in range(0,5)]

In [201]:
class GainDataGenerator(keras.utils.Sequence):
    'Generates data for GAIN'
    def __init__(self,
                 data_list,
                 batch_size=32,
                 input_width=24*3,
                 label_width=24*3,
                 shift=0,
                 fill_no=4,
                 miss_rate=0.2,
                 hint_rate=0.9,
                 normalize=True,
                 miss_pattern=None,
                 miss_data_load_dir='save',
                 alpha=100.):
        'Initialization'
        window_size = input_width
        
        # interpollation
        filled_data = []
        for data in data_list:
            data = interpolate(data, max_gap=fill_no)
            filled_data.append(data)
            
        data_list = filled_data
        
        # whole data
        self.data = np.concatenate(data_list)

        # TO-DO
        
        # pre calculation for  sequence data
        last_cum = 0
        cums = []
        for data in data_list:
            isnan = np.isnan(data)
            isany = np.any(isnan, axis=1)
            #shift same as pd.shift(isany, fill_value=True)
            shifted = np.roll(isany, 1)
            shifted[0] = True # set to nan
            
            start_seq = ((isany == False) & (shifted == True)).astype(int)
            cum = start_seq.cumsum()
            cum += last_cum
            last_cum = np.max(cum)
            cum[isany] = 0
            cums.append(cum)

        # Define mask matrix
        if miss_pattern is None:
            print("pattern none")
            self.data_m = binary_sampler(1-miss_rate, self.data.shape)
        else:
            #MissData.save(self.data, max_tseq = 12)
            print("load save")
            self.miss = MissData(load_dir=miss_data_load_dir)
            self.miss_rate = miss_rate
            miss_data = self.miss.make_missdata(self.data, self.miss_rate)
            self.data_m = 1. - np.isnan(miss_data).astype(float)
            
            self.data_m_rand = binary_sampler(1-(miss_rate/10.), self.data.shape)
            self.data_m[self.data_m_rand==0.] = 0.
        self.miss_pattern = miss_pattern
        
        # sequence data
        self.ids = np.concatenate(cums)
        data_idx = np.empty((0), dtype=int)
        for i in range(1, last_cum+1):
            seq_len = (self.ids == i).sum()
            start_id = np.argmax(self.ids == i)
            # possible data number in seqeunce
            time_len = seq_len - window_size + 1
            start_ids = np.arange(start_id, start_id+time_len)
            data_idx = np.append(data_idx, start_ids)
            
        # start index set for sequence data
        self.data_idx = data_idx
        self.input_width = input_width
        self.no = len(data_idx)
        
        #print('self.no = ', self.no)
        
        self.batch_size = batch_size
        
        # random shuffling  index
        self.batch_idx = sample_batch_index(self.no, self.no)
        self.batch_id = 0
        self.shape = (batch_size,self.input_width)+self.data.shape[1:]
        #self.hint_rate = hint_rate
            
    def __len__(self):
        'Denotes the number of batches per epoch'
        #return int(128/self.batch_size)
        #return 2
        return 1

    def __getitem__(self, index):
        'Generate one batch of data'
        #print('index =', index)
        # Sample batch
        x = np.empty((0, self.input_width, self.data.shape[1]))
        #m = np.empty((0, self.input_width, self.data.shape[1]))
        #h = np.empty((0, self.input_width, self.data.shape[1]))
        y = np.empty((0, self.input_width, self.data.shape[1]))
        #print(x.shape)
        #print(self.data.shape)
        #print(self.input_width)
        #self.batch_idx = sample_batch_index(self.no, self.batch_size)
        for cnt in range(0, self.batch_size):
            i = self.batch_idx[self.batch_id]
            self.batch_id += 1
            #self.batch_id %= self.batch_size
            self.batch_id %= self.no
            if self.miss_pattern and (self.batch_id == 0):
                self.batch_idx = sample_batch_index(self.no, self.no)
                miss_data = self.miss.make_missdata(self.data, self.miss_rate)
                self.data_m = 1. - np.isnan(miss_data).astype(float)
                self.data_m_rand = binary_sampler(1-self.miss_rate/10., self.data.shape)
                self.data_m[self.data_m_rand==0.] = 0.
            idx1 = self.data_idx[i]
            idx2 = self.data_idx[i]+self.input_width
            #print(idx1, idx2)
        
            Y_mb = self.data[idx1:idx2].copy()
            X_mb = Y_mb.copy()
            M_mb = self.data_m[idx1:idx2]
            Z_mb = uniform_sampler(0, 0.01, shape=X_mb.shape)
            X_mb = M_mb*X_mb + (1-M_mb)*Z_mb
            #H_mb_temp = binary_sampler(self.hint_rate, shape=X_mb.shape)
            #H_mb = M_mb * H_mb_temp
            X_mb[M_mb == 0] = np.nan
            Y_mb[M_mb == 1] = np.nan
            x = np.append(x, [X_mb], axis=0)
            #m = np.append(m, [M_mb], axis=0)
            #h = np.append(h, [H_mb], axis=0)
            y = np.append(y, [Y_mb], axis=0)
            
        #return [x, m, h], y
        return x, y
    
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        return

# dgen = GainDataGenerator(df)

In [202]:
class GAIN(keras.Model):
    def __init__(self, shape, alpha=100., load=False, hint_rate=0.9, gen_sigmoid=True, **kwargs):
        super(GAIN, self).__init__(**kwargs)
        self.shape = shape
        self.dim = np.prod(shape).astype(int)
        self.h_dim = self.dim
        self.gen_sigmoid = gen_sigmoid
        self.build_generator()
        self.build_discriminator()
        self.hint_rate = hint_rate
        self.alpha = alpha
        self.generator_optimizer = Adam()
        self.discriminator_optimizer = Adam()

    ## GAIN models
    def build_generator(self):
        last_activation = 'sigmoid' if self.gen_sigmoid else None
        xavier_initializer = tf.keras.initializers.GlorotNormal()

        shape = self.shape
        #x = Input(shape=(self.dim,), name='generator_input_x')
        #m = Input(shape=(self.dim,), name='generator_input_m')
        x = Input(shape=shape, name='generator_input_x')
        m = Input(shape=shape, name='generator_input_m')
        
        x_f = Flatten()(x)
        m_f = Flatten()(m)

        a = Concatenate()([x_f, m_f])

        a = Dense(self.h_dim, activation='relu', kernel_initializer=xavier_initializer)(a)
        #a = keras.layers.BatchNormalization()(a)
        a = Dense(self.h_dim, activation='relu', kernel_initializer=xavier_initializer)(a)
        #a = keras.layers.BatchNormalization()(a)
        a = Dense(self.dim, activation=last_activation, kernel_initializer=xavier_initializer)(a)
        G_prob = keras.layers.Reshape(shape)(a)
        self.generator = keras.models.Model([x, m], G_prob, name='generator')

    def build_discriminator(self):
        xavier_initializer = tf.keras.initializers.GlorotNormal()
        shape = self.shape

        #x = Input(shape=(self.dim,), name='discriminator_input_x')
        #h = Input(shape=(self.dim,), name='discriminator_input_h')
        x = Input(shape=shape, name='discriminator_input_x')
        h = Input(shape=shape, name='discriminator_input_h')
        
        x_f = Flatten()(x)
        h_f = Flatten()(h)

        a = Concatenate()([x_f, h_f])

        a = Dense(self.h_dim, activation='relu', kernel_initializer=xavier_initializer)(a)
        a = Dense(self.h_dim, activation='relu', kernel_initializer=xavier_initializer)(a)
        a = Dense(self.dim, activation='sigmoid', kernel_initializer=xavier_initializer)(a)
        D_prob = keras.layers.Reshape(shape)(a)
        self.discriminator = keras.models.Model([x, h], D_prob, name='discriminator')
        
    def call(self, inputs):
        if isinstance(inputs, tuple):
            inputs = inputs[0]
        shape = inputs.shape
        dims = np.prod(shape[1:])
        input_width = shape[1]
        #print('inputs.shape=',inputs.shape)
        x = inputs
        #x = x.reshape((n, -1))
        #print('dims=',dims)
        #x = keras.layers.Reshape((dims,))(x)
        #x = keras.layers.Reshape(tf.TensorShape((self.dim,)))(x)
        #print('x =', x)
        #print('x.shape = ', x.shape)
        #x = keras.layers.Reshape(tf.TensorShape([57]))(x)
        
        isnan = tf.math.is_nan(x)
        #m = 1.- keras.backend.cast(isnan, dtype=tf.float32)
        m = tf.where(isnan, 0., 1.)
        z = keras.backend.random_uniform(shape=tf.shape(x), minval=0.0, maxval=0.01)
        x = tf.where(isnan, z, x)
        #z = uniform_sampler(0, 0.01, shape=x.shape)
        #z = tf.keras.backend.random_uniform(shape=x.shape, minval=0.0, maxval=0.01)
        imputed_data = self.generator([x, m], training=False)
        #imputed_data = m*x + (1-m)*imputed_data
        #imputed_data = tf.where(isnan, imputed_data, np.nan)
        imputed_data = tf.where(isnan, imputed_data, x)
        #imputed_data = keras.layers.Reshape(shape[1:])(imputed_data)
        #print('imputed_data.shape = ', imputed_data.shape)
        
        return imputed_data
    
    def D_loss(M, D_prob):
        ## GAIN loss
        return -tf.reduce_mean(M * tf.keras.backend.log(D_prob + 1e-8) \
                         + (1-M) * tf.keras.backend.log(1. - D_prob + 1e-8))
    
    def G_loss(self, M, D_prob, X, G_sample):
        G_loss_temp = -tf.reduce_mean((1-M) * tf.keras.backend.log(D_prob + 1e-8))
        MSE_loss = tf.reduce_mean((M * X - M * G_sample)**2) / (tf.reduce_mean(M) + 1e-8)
        #G_loss_temp = GAIN.G_loss_bincross(M, D_prob)
        #MSE_loss = GAIN.MSE_loss(M, X, G_sample)
        G_loss = G_loss_temp + self.alpha * MSE_loss
        return G_loss
        
    def RMSE_loss(y_true, y_pred):
        isnan = tf.math.is_nan(y_true)
        M = tf.where(isnan, 1., 0.)
        return tf.sqrt(tf.reduce_sum(tf.where(isnan, 0., y_pred-y_true)**2)/tf.reduce_sum(1-M))
    
    def train_step(self, data):
        #[x, m, h], y = data
        x, y = data
        #X = keras.layers.Reshape((self.dim,), input_shape=self.shape)(x)
        #Y = keras.layers.Reshape((self.dim,), input_shape=self.shape)(y)
        #X = keras.layers.Flatten()(x)
        #Y = keras.layers.Flatten()(y)
        X = x
        Y = y
        #X = tf.reshape(x, shape=(x.shape[0], -1))
        #Y = tf.reshape(y, shape=(x.shape[0], -1))
        isnan = tf.math.is_nan(X)
        #M = 1 - keras.backend.cast(isnan, dtype=tf.float32)
        M = tf.where(isnan, 0., 1.)
        Z = keras.backend.random_uniform(shape=tf.shape(X), minval=0.0, maxval=0.01)
        #H_temp = binary_sampler(self.hint_rate, shape=X.shape)
        H_rand = keras.backend.random_uniform(shape=tf.shape(X), minval=0.0, maxval=1.)
        #H_temp = 1*keras.backend.cast((H_rand < self.hint_rate), dtype=tf.float32)
        H_temp = tf.where(H_rand < self.hint_rate, 1., 0.)
        
        H = M * H_temp
        #X = M * X + (1-M) * Z
        X = tf.where(isnan, Z, X)
        with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
            G_sample = self.generator([X, M], training=True)

            # Combine with observed data
            #Hat_X = tf.where(isnan, G_sample, X)
            Hat_X = X * M + G_sample * (1-M)
            D_prob = self.discriminator([Hat_X, H], training=True)
            gen_loss = self.G_loss(M, D_prob, X, G_sample)
            disc_loss = tf.keras.backend.mean(tf.keras.losses.binary_crossentropy(M, D_prob))
            #disc_loss = GAIN.D_loss(M, D_prob)
            #disc_loss = GAIN.D_loss(M, D_prob)

        gradients_of_generator = gen_tape.gradient(gen_loss, self.generator.trainable_variables)
        gradients_of_discriminator = disc_tape.gradient(disc_loss, self.discriminator.trainable_variables)

        self.generator_optimizer.apply_gradients(zip(gradients_of_generator, self.generator.trainable_variables))
        self.discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, self.discriminator.trainable_variables))
        
        rmse = tf.sqrt(tf.reduce_sum(tf.where(isnan, G_sample - Y, 0.)**2)/tf.reduce_sum(1-M))
        return {
                 'gen_loss':     gen_loss,
                 'disc_loss':    disc_loss,
                 'rmse':         rmse,
               }
    
    def save(self, save_dir='save'):
        if not os.path.exists(save_dir):
          os.makedirs(save_dir)
        disc_savefile = os.path.join(save_dir, 'discriminator.h5')
        gen_savefile = os.path.join(save_dir, 'generator.h5')
        self.discriminator.save_weights(disc_savefile)
        self.generator.save_weights(gen_savefile)

    def load(self, save_dir='save'):
        disc_savefile = os.path.join(save_dir, 'discriminator.h5')
        gen_savefile = os.path.join(save_dir, 'generator.h5')
        try:
          self.discriminator.load_weights(disc_savefile)
          self.generator.load_weights(gen_savefile)
          print('model weights loaded')
        except:
          print('model loadinng error')

In [203]:
def make_dataset_gain(self, data):
  dg = GainDataGenerator(
      df,
      input_width = self.input_width,
      label_width = self.label_width,
      batch_size = 128,
      normalize = False,
      miss_pattern = True,
      miss_rate = 0.15,
      fill_no = 3,
  )
  self.dg = dg
  ds = tf.data.Dataset.from_generator(
      lambda: dg,
      output_types=(tf.float32, tf.float32),
      output_shapes=(
        dg.shape,
        dg.shape
        #[batch_size, train_generator.dim],
        #[batch_size, train_generator.dim],
      )
  )
  return ds

WindowGenerator.make_dataset = make_dataset_gain

# 측정망별 반복 구간 

In [204]:
norm_df = pd.concat(df,axis=0)
norm_data = norm_df.to_numpy()
MissData.save(norm_data, max_tseq = 24)
# 1: 자동, 2: 수질, 

[[    0    14     2     0]
 [    2     1     1    14]
 [    3     4     1    15]
 ...
 [13689    15     3 46636]
 [13692     7     1 46651]
 [13693    14     3 46658]]
miss_data file saved


In [205]:
dgen = GainDataGenerator(df)

pattern none


In [206]:
train_df = df_all
val_df = df_all
test_df = df_all
df_all

,tmpr_value,ph_value,do_value,ec_value,toc_value,총질소_값,총인_값,클로로필-a_값,Day sin,Day cos,Year sin,Year cos
0,3.32167,6.75808,12.03583,271.90833,1.94923,NaN,NaN,12.15102,-8.039063e-12,1.000000,0.009848,0.999952
1,3.11642,6.74925,12.03242,272.30000,1.92317,NaN,NaN,11.98576,2.588190e-01,0.965926,0.010565,0.999944
2,2.95050,6.74333,12.03525,273.09167,1.88344,NaN,NaN,12.11111,5.000000e-01,0.866025,0.011282,0.999936
3,2.95350,6.74450,12.03333,274.91667,1.92978,NaN,NaN,12.22508,7.071068e-01,0.707107,0.011999,0.999928
4,2.87008,6.74250,12.04175,275.99167,1.94525,NaN,NaN,12.12821,8.660254e-01,0.500000,0.012715,0.999919
...,...,...,...,...,...,...,...,...,...,...,...,...
65083,27.80833,7.00000,5.40000,212.00000,6.72250,2.39225,0.13117,7.24750,-9.659258e-01,0.258819,0.447255,-0.894407
65084,27.45000,7.00000,4.97583,208.50000,6.78500,2.39750,0.13008,7.17417,-8.660254e-01,0.500000,0.446614,-0.894727
65085,27.09167,7.00000,4.51333,210.33333,6.44000,2.45700,0.12908,7.01167,-7.071068e-01,0.707107,0.445972,-0.895047
65086,26.67500,6.95833,4.07333,215.58333,5.54667,2.42975,0.12900,6.95750,-5.000000e-01,0.866025,0.445330,-0.895366


In [207]:
wide_window = WindowGenerator(
    input_width=24*5, label_width=24*5, shift=0,
    #label_columns=['T (degC)']
)

wide_window

Total window size: 120
Input indices: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119]
Label indices: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 

## Graph 

In [208]:
wide_window.example[0].shape

load save


TensorShape([128, 120, 12])

In [209]:
#wide_window.plot(plot_col='toc_value')
#print('make_dataset_gain: dg.no = ', wide_window.dg.no)

In [210]:
plt.figure(figsize=(9,10))
isnan = np.isnan(norm_data).astype(int)
data = isnan
n = data.shape[0]
seq_len = n//8
for i in range(8):
    plt.subplot(181+i)
    plt.imshow(data[i*seq_len:(i+1)*seq_len, 0:], aspect='auto')
    plt.yticks([])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## 컴파일 및 학습_GAIN

In [211]:
val_performance = {}
performance = {}

In [212]:
gain = GAIN(shape=wide_window.dg.shape[1:], gen_sigmoid=False)
gain.compile(loss=GAIN.RMSE_loss)

In [213]:
MAX_EPOCHS = 2000

def compile_and_fit(model, window, patience=10):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')

  #model.compile(loss=tf.losses.MeanSquaredError(),
                #optimizer=tf.optimizers.Adam(),
                #metrics=[tf.metrics.MeanAbsoluteError()])
  model.compile(loss=GAIN.RMSE_loss)

  history = model.fit(window.train, epochs=MAX_EPOCHS,
                      validation_data=window.val,
                      callbacks=[early_stopping])
  return history

**모델 학습**

In [214]:
history = compile_and_fit(gain, wide_window, patience=MAX_EPOCHS//5)


val_performance['Gain'] = gain.evaluate(wide_window.val)
performance['Gain'] = gain.evaluate(wide_window.test, verbose=0)

# gain.save(save_dir='save')

load save
load save
Epoch 1/2000
1/1 [==============================] - 0s 182ms/step - gen_loss: 123.3129 - disc_loss: 0.7309 - rmse: 1.0675 - val_loss: 1.0477
Epoch 2/2000
1/1 [==============================] - 0s 31ms/step - gen_loss: 98.1995 - disc_loss: 0.5214 - rmse: 0.9635 - val_loss: 0.9225
Epoch 3/2000
1/1 [==============================] - 0s 31ms/step - gen_loss: 84.6110 - disc_loss: 0.4829 - rmse: 0.8947 - val_loss: 0.8249
Epoch 4/2000
1/1 [==============================] - 0s 31ms/step - gen_loss: 72.9531 - disc_loss: 0.3869 - rmse: 0.8426 - val_loss: 0.8485
Epoch 5/2000
1/1 [==============================] - 0s 31ms/step - gen_loss: 66.5823 - disc_loss: 0.3693 - rmse: 0.8166 - val_loss: 0.7894
Epoch 6/2000
1/1 [==============================] - 0s 31ms/step - gen_loss: 57.1950 - disc_loss: 0.3803 - rmse: 0.7785 - val_loss: 0.7410
Epoch 7/2000
1/1 [==============================] - 0s 31ms/step - gen_loss: 47.0412 - disc_loss: 0.3698 - rmse: 0.7075 - val_loss: 0.6628
Epoch

1/1 [==============================] - 0s 32ms/step - gen_loss: 6.8460 - disc_loss: 0.2931 - rmse: 0.3400 - val_loss: 0.3509
Epoch 60/2000
1/1 [==============================] - 0s 32ms/step - gen_loss: 7.1120 - disc_loss: 0.2976 - rmse: 0.3550 - val_loss: 0.3568
Epoch 61/2000
1/1 [==============================] - 0s 31ms/step - gen_loss: 7.4668 - disc_loss: 0.2976 - rmse: 0.3504 - val_loss: 0.3455
Epoch 62/2000
1/1 [==============================] - 0s 32ms/step - gen_loss: 6.4744 - disc_loss: 0.3009 - rmse: 0.3391 - val_loss: 0.3541
Epoch 63/2000
1/1 [==============================] - 0s 31ms/step - gen_loss: 6.9149 - disc_loss: 0.2825 - rmse: 0.3502 - val_loss: 0.3387
Epoch 64/2000
1/1 [==============================] - 0s 32ms/step - gen_loss: 7.2331 - disc_loss: 0.2941 - rmse: 0.3421 - val_loss: 0.3479
Epoch 65/2000
1/1 [==============================] - 0s 31ms/step - gen_loss: 7.1911 - disc_loss: 0.2848 - rmse: 0.3691 - val_loss: 0.3715
Epoch 66/2000
1/1 [======================

Epoch 118/2000
1/1 [==============================] - 0s 32ms/step - gen_loss: 5.2129 - disc_loss: 0.2334 - rmse: 0.3284 - val_loss: 0.3266
Epoch 119/2000
1/1 [==============================] - 0s 32ms/step - gen_loss: 4.9114 - disc_loss: 0.2298 - rmse: 0.3007 - val_loss: 0.3348
Epoch 120/2000
1/1 [==============================] - 0s 32ms/step - gen_loss: 4.6604 - disc_loss: 0.2232 - rmse: 0.2969 - val_loss: 0.3502
Epoch 121/2000
1/1 [==============================] - 0s 32ms/step - gen_loss: 4.8844 - disc_loss: 0.2268 - rmse: 0.3065 - val_loss: 0.3175
Epoch 122/2000
1/1 [==============================] - 0s 31ms/step - gen_loss: 4.9600 - disc_loss: 0.2223 - rmse: 0.3138 - val_loss: 0.3330
Epoch 123/2000
1/1 [==============================] - 0s 30ms/step - gen_loss: 4.8483 - disc_loss: 0.2297 - rmse: 0.3056 - val_loss: 0.3683
Epoch 124/2000
1/1 [==============================] - 0s 30ms/step - gen_loss: 5.0917 - disc_loss: 0.2282 - rmse: 0.3265 - val_loss: 0.3123
Epoch 125/2000
1/1 [

1/1 [==============================] - 0s 32ms/step - gen_loss: 4.2981 - disc_loss: 0.2051 - rmse: 0.2911 - val_loss: 0.3443
Epoch 177/2000
1/1 [==============================] - 0s 31ms/step - gen_loss: 4.6022 - disc_loss: 0.2152 - rmse: 0.3079 - val_loss: 0.3279
Epoch 178/2000
1/1 [==============================] - 0s 31ms/step - gen_loss: 4.3709 - disc_loss: 0.2054 - rmse: 0.3024 - val_loss: 0.3343
Epoch 179/2000
1/1 [==============================] - 0s 32ms/step - gen_loss: 4.4443 - disc_loss: 0.2098 - rmse: 0.3045 - val_loss: 0.3298
Epoch 180/2000
1/1 [==============================] - 0s 32ms/step - gen_loss: 4.6134 - disc_loss: 0.2137 - rmse: 0.3154 - val_loss: 0.2981
Epoch 181/2000
1/1 [==============================] - 0s 31ms/step - gen_loss: 4.0540 - disc_loss: 0.2070 - rmse: 0.2772 - val_loss: 0.3152
Epoch 182/2000
1/1 [==============================] - 0s 32ms/step - gen_loss: 4.9669 - disc_loss: 0.2137 - rmse: 0.3287 - val_loss: 0.3316
Epoch 183/2000
1/1 [===============

1/1 [==============================] - 0s 30ms/step - gen_loss: 4.0321 - disc_loss: 0.1880 - rmse: 0.2905 - val_loss: 0.3167
Epoch 235/2000
1/1 [==============================] - 0s 31ms/step - gen_loss: 4.5649 - disc_loss: 0.1949 - rmse: 0.3033 - val_loss: 0.3040
Epoch 236/2000
1/1 [==============================] - 0s 31ms/step - gen_loss: 4.2295 - disc_loss: 0.1933 - rmse: 0.3004 - val_loss: 0.3065
Epoch 237/2000
1/1 [==============================] - 0s 31ms/step - gen_loss: 4.2976 - disc_loss: 0.1948 - rmse: 0.2996 - val_loss: 0.3064
Epoch 238/2000
1/1 [==============================] - 0s 31ms/step - gen_loss: 4.4890 - disc_loss: 0.1879 - rmse: 0.2990 - val_loss: 0.3317
Epoch 239/2000
1/1 [==============================] - 0s 30ms/step - gen_loss: 4.5511 - disc_loss: 0.1822 - rmse: 0.2818 - val_loss: 0.2985
Epoch 240/2000
1/1 [==============================] - 0s 30ms/step - gen_loss: 4.0463 - disc_loss: 0.1837 - rmse: 0.2731 - val_loss: 0.3169
Epoch 241/2000
1/1 [===============

...

1/1 [==============================] - 0s 30ms/step - gen_loss: 2.6818 - disc_loss: 0.1361 - rmse: 0.2913 - val_loss: 0.3172
Epoch 1047/2000
1/1 [==============================] - 0s 30ms/step - gen_loss: 2.4870 - disc_loss: 0.1373 - rmse: 0.2759 - val_loss: 0.3246
Epoch 1048/2000
1/1 [==============================] - 0s 30ms/step - gen_loss: 2.6506 - disc_loss: 0.1388 - rmse: 0.2912 - val_loss: 0.2896
Epoch 1049/2000
1/1 [==============================] - 0s 30ms/step - gen_loss: 2.6982 - disc_loss: 0.1296 - rmse: 0.2607 - val_loss: 0.3292
Epoch 1050/2000
1/1 [==============================] - 0s 32ms/step - gen_loss: 2.5704 - disc_loss: 0.1353 - rmse: 0.2735 - val_loss: 0.2608
Epoch 1051/2000
1/1 [==============================] - 0s 32ms/step - gen_loss: 2.8304 - disc_loss: 0.1338 - rmse: 0.3035 - val_loss: 0.2949
Epoch 1052/2000
1/1 [==============================] - 0s 33ms/step - gen_loss: 2.5743 - disc_loss: 0.1339 - rmse: 0.2946 - val_loss: 0.3156
Epoch 1053/2000
1/1 [========

1/1 [==============================] - 0s 31ms/step - gen_loss: 2.5319 - disc_loss: 0.1339 - rmse: 0.2908 - val_loss: 0.2796
Epoch 1105/2000
1/1 [==============================] - 0s 31ms/step - gen_loss: 2.3688 - disc_loss: 0.1265 - rmse: 0.2698 - val_loss: 0.2919
Epoch 1106/2000
1/1 [==============================] - 0s 31ms/step - gen_loss: 2.6383 - disc_loss: 0.1331 - rmse: 0.2786 - val_loss: 0.2689
Epoch 1107/2000
1/1 [==============================] - 0s 31ms/step - gen_loss: 2.5086 - disc_loss: 0.1336 - rmse: 0.2826 - val_loss: 0.2922
Epoch 1108/2000
1/1 [==============================] - 0s 31ms/step - gen_loss: 2.4060 - disc_loss: 0.1308 - rmse: 0.2653 - val_loss: 0.2838
Epoch 1109/2000
1/1 [==============================] - 0s 31ms/step - gen_loss: 2.4322 - disc_loss: 0.1314 - rmse: 0.2947 - val_loss: 0.3017
Epoch 1110/2000
1/1 [==============================] - 0s 31ms/step - gen_loss: 2.3686 - disc_loss: 0.1354 - rmse: 0.2662 - val_loss: 0.2763
Epoch 1111/2000
1/1 [========

1/1 [==============================] - 0s 30ms/step - gen_loss: 2.2791 - disc_loss: 0.1280 - rmse: 0.2754 - val_loss: 0.2895
Epoch 1163/2000
1/1 [==============================] - 0s 31ms/step - gen_loss: 2.3747 - disc_loss: 0.1300 - rmse: 0.2573 - val_loss: 0.2927
Epoch 1164/2000
1/1 [==============================] - 0s 30ms/step - gen_loss: 2.5764 - disc_loss: 0.1264 - rmse: 0.3066 - val_loss: 0.3048
Epoch 1165/2000
1/1 [==============================] - 0s 31ms/step - gen_loss: 2.4489 - disc_loss: 0.1324 - rmse: 0.2785 - val_loss: 0.3112
Epoch 1166/2000
1/1 [==============================] - 0s 381ms/step - gen_loss: 2.5581 - disc_loss: 0.1317 - rmse: 0.2803 - val_loss: 0.3673
Epoch 1167/2000
1/1 [==============================] - 0s 30ms/step - gen_loss: 2.7429 - disc_loss: 0.1342 - rmse: 0.2868 - val_loss: 0.2894
Epoch 1168/2000
1/1 [==============================] - 0s 32ms/step - gen_loss: 2.8000 - disc_loss: 0.1330 - rmse: 0.3008 - val_loss: 0.2797
Epoch 1169/2000
1/1 [=======

1/1 [==============================] - 0s 31ms/step - gen_loss: 2.2757 - disc_loss: 0.1296 - rmse: 0.2749 - val_loss: 0.2862
Epoch 1221/2000
1/1 [==============================] - 0s 31ms/step - gen_loss: 2.3800 - disc_loss: 0.1298 - rmse: 0.2736 - val_loss: 0.2889
Epoch 1222/2000
1/1 [==============================] - 0s 31ms/step - gen_loss: 2.4461 - disc_loss: 0.1296 - rmse: 0.2631 - val_loss: 0.2815
Epoch 1223/2000
1/1 [==============================] - 0s 31ms/step - gen_loss: 2.6062 - disc_loss: 0.1304 - rmse: 0.2708 - val_loss: 0.3101
Epoch 1224/2000
1/1 [==============================] - 0s 32ms/step - gen_loss: 2.5901 - disc_loss: 0.1302 - rmse: 0.2824 - val_loss: 0.2943
Epoch 1225/2000
1/1 [==============================] - 0s 31ms/step - gen_loss: 2.4799 - disc_loss: 0.1317 - rmse: 0.2874 - val_loss: 0.3014
Epoch 1226/2000
1/1 [==============================] - 0s 32ms/step - gen_loss: 2.3977 - disc_loss: 0.1274 - rmse: 0.2723 - val_loss: 0.2981
Epoch 1227/2000
1/1 [========

1/1 [==============================] - 0s 31ms/step - gen_loss: 2.4026 - disc_loss: 0.1270 - rmse: 0.2733 - val_loss: 0.2920
Epoch 1279/2000
1/1 [==============================] - 0s 31ms/step - gen_loss: 2.5406 - disc_loss: 0.1268 - rmse: 0.2759 - val_loss: 0.3095
Epoch 1280/2000
1/1 [==============================] - 0s 31ms/step - gen_loss: 2.6619 - disc_loss: 0.1246 - rmse: 0.2861 - val_loss: 0.3102
Epoch 1281/2000
1/1 [==============================] - 0s 31ms/step - gen_loss: 2.4494 - disc_loss: 0.1284 - rmse: 0.2780 - val_loss: 0.3278
Epoch 1282/2000
1/1 [==============================] - 0s 31ms/step - gen_loss: 2.4836 - disc_loss: 0.1278 - rmse: 0.2746 - val_loss: 0.2993
Epoch 1283/2000
1/1 [==============================] - 0s 31ms/step - gen_loss: 2.4038 - disc_loss: 0.1280 - rmse: 0.2604 - val_loss: 0.2904
load save
1/1 [==============================] - 0s 5ms/step - loss: 0.2862
load save


**학습 loss history 출력**

In [215]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax2 = ax.twinx()
ax.plot(history.history['gen_loss'], label='gen_loss')
ax.plot(history.history['disc_loss'], label='disc_loss')
ax2.plot(history.history['rmse'], label='rmse', color='green')
ax2.plot(history.history['val_loss'], label='val_loss', color='red')
#plt.legend(history.history.keys(), loc='upper right')
#ax.legend(loc='upper center')
ax.legend(loc='upper center')
ax2.legend(loc='upper right')
ax.set_xlabel("epochs")
ax.set_ylabel("loss")
ax2.set_ylabel("rmse")
plt.show()    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**성능 측정**

In [216]:
gain.evaluate(wide_window.test.repeat(), steps=100)

load save
100/100 [==============================] - 2s 15ms/step - loss: 0.3039


0.3039109706878662

**model save**

In [217]:
gain.save(save_dir='save')

**샘플 prediction 출력**

In [218]:
wide_window.plot(gain, plot_col='do_value')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [219]:
wide_window.plot(gain, plot_col='toc_value')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [220]:
wide_window.plot(gain, plot_col='총질소_값')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [221]:
wide_window.plot(gain, plot_col='총인_값')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [222]:
wide_window.plot(gain, plot_col='클로로필-a_값')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [223]:
norm_df = pd.concat(df,axis=0)

In [224]:
data = norm_df.to_numpy()

total_n = wide_window.dg.data.shape[0]
print(total_n)
unit_shape = wide_window.dg.shape[1:]
print(unit_shape)
#dim = np.prod(wide_window.dg.shape[1:]).astype(int)
dim = wide_window.dg.shape[1]
print(dim)
n = (total_n//dim)*dim

x = data[0:n].copy()
y_true = data[0:n].copy()

#x = interpolate(x, max_gap=3)

print('x.shape =', x.shape)
x_reshape = x.reshape((-1,)+unit_shape)
print('x_reshape.shape =', x_reshape.shape)
isnan = np.isnan(x_reshape)
print(isnan.sum())
print('y_true.shape=', y_true.shape)
isnan = np.isnan(y_true)
print(isnan.sum())

x_remain = data[-wide_window.dg.shape[1]:].copy()
x_remain_reshape = x_remain.reshape((-1,)+unit_shape)
x_remain_reshape.shape

195264
(120, 12)
120
x.shape = (195240, 12)
x_reshape.shape = (1627, 120, 12)
281190
y_true.shape= (195240, 12)
281190


(1, 120, 12)

In [225]:
gain.evaluate(x_reshape, y_true.reshape((-1,)+unit_shape))

51/51 [==============================] - 0s 1ms/step - loss: 0.0000e+00


0.0

In [226]:
y_pred = gain.predict(x_reshape)
y_remain_pred = gain.predict(x_remain_reshape)

In [227]:
y_pred = y_pred.reshape(y_true.shape)
y_remain_pred = y_remain_pred.reshape(x_remain.shape)
print(y_pred.shape, y_remain_pred.shape)
y_pred = np.append(y_pred, y_remain_pred[-(total_n-n):], axis=0)
print(y_pred.shape)

(195240, 12) (120, 12)
(195264, 12)


In [228]:
y_pred[~np.isnan(data)] = np.nan

In [229]:
n = 8
plt.figure(figsize=(9,20))
for i in range(n):
    #plt.subplot('%d1%d'%(n,i))
    plt.subplot(811+i)
    plt.plot(x[:, i])
    plt.plot(y_pred[:, i])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**Imputed Data**

In [ ]:
total_n = wide_window.dg.data.shape[0]
print(total_n)
unit_shape = wide_window.dg.shape[1:]
print('unit_shape=', unit_shape)
time_seq = unit_shape[0]
print(time_seq)
n = (total_n//time_seq)*time_seq
print('n=', n)

gans = []
oris = []
for i in range(len(df)):
    x = df[i].to_numpy()
    total_n = x.shape[0]
    n = (total_n//time_seq)*time_seq
    x = x[0:n]
    x_block = x.reshape((-1,)+unit_shape)
    y = gain.predict(x_block)
    y_gan = y.reshape(x.shape)
    
    # cut off sin, cos data
    if (i > 0):
        x = x[:, :-4]
        y_gan = y_gan[:, :-4]
    gans.append(y_gan)
    oris.append(x)
print(x.shape)
print(y_gan.shape)

# 예측

In [ ]:
df[0].head

In [ ]:
ori = np.concatenate(oris, axis=1)
gan = np.concatenate(gans, axis=1)
# print(oris[0].shape, gans[0].shape)
# print(oris[1].shape, gans[1].shape)
# print(ori.shape, gan.shape)

total_no = ori.shape[0]
train_no = int(total_no*0.7)

train_slice = slice(0, train_no)
val_slice = slice(train_no, None)
test_slice = slice(0, None)

train_df = pd.DataFrame(gan[train_slice])
val_df = pd.DataFrame(gan[val_slice])
test_df = pd.DataFrame(gan[test_slice])

train_ori_df = pd.DataFrame(ori[train_slice])
val_ori_df = pd.DataFrame(ori[val_slice])
test_ori_df = pd.DataFrame(ori[test_slice])

num_features = train_df.shape[1]
#out_num_features = oris[0].shape[1]-4
# out_features = [target_col_idx]
# out_num_features = len(out_features)
# out_num_features

In [ ]:
train_df 

In [ ]:
df[0].columns

In [ ]:
MAX_EPOCHS = 150
target_col = '총유기탄소'
num_features = df[0].shape[1]
label_columns_indices = {name: i for i, name in enumerate(df[0])}
target_col_idx = label_columns_indices[target_col]
# target_col_idx
out_features = [target_col_idx]
# out_features = [3,4,5,6,7]
out_num_features = len(out_features)
out_num_features

In [ ]:
out_features

In [ ]:
class WaterDataGenerator(keras.utils.Sequence):
    'Generates data for water'
    def __init__(self,
                 imputed_data,
                 ori_data = None,
                 batch_size=32,
                 input_width=24*7,
                 label_width=24*3,
                 shift=24*3,
                 skip_time = None,
                 shuffle = True,
                 out_features = None,
                 out_num_features = None,
                ):
        'Initialization'
        self.window_size = input_width+shift
        self.total_no = imputed_data.shape[0]
        self.data = imputed_data
        self.input_width = input_width
        self.label_width = label_width
        self.batch_size = batch_size
        self.input_shape = (batch_size, input_width, self.data.shape[1])
        self.out_num_features = out_num_features
#         print("out_features")
#         print(out_features)
        if out_features:
            self.out_features = out_features
        else:
            self.out_features = [i for i in range(out_num_features)]
        self.label_shape = (batch_size, label_width, self.out_num_features)
        if (skip_time):
            # TO-DO
            self.no = self.total_no - self.window_size
            self.data_idx = np.arange(0, self.no)
        else:
            self.no = self.total_no - self.window_size
            self.data_idx = np.arange(0, self.no)
            
        if shuffle:
            self.batch_idx = np.random.permutation(self.no)
        else:
            self.batch_idx = np.arange(0, self.no)
        self.batch_id = 0
        
        
    def __len__(self):
        'Denotes the number of batches per epoch'
        #return int(128/self.batch_size)
        #return 2
        return 1

    def __getitem__(self, index):
        'Generate one batch of data'
        #print('index =', index)
        #print('self.no =', self.no)
        #print('self.total_no =', self.total_no)
        #print('self.batch_id =', self.batch_id)
        # Sample batch
        label_width = self.label_width
        batch_idx = self.batch_idx
        
        x = np.empty((0, self.input_width, self.data.shape[1]))
        
        #y = np.empty((0, self.input_width, self.data.shape[1]))
        y = np.empty((0, self.label_width, self.out_num_features))
       
        #print(x.shape)
        #print(y.shape)
        
        
        for cnt in range(0, self.batch_size):
            i = self.batch_id
            self.batch_id += 1
            idx1 = self.data_idx[batch_idx[i]]
            idx2 = idx1 + self.input_width
            
            X = self.data[idx1:idx2].to_numpy()
            
            
            idx1 = self.data_idx[batch_idx[i]] + self.window_size - label_width
            idx2 = idx1 + label_width
            
            #Y = self.data[idx1:idx2,:,:out_num_features]
            Y = self.data.iloc[idx1:idx2, self.out_features].to_numpy()
            #Y = self.data[idx1:idx2]
            #print('Y.shape = ', Y.shape)
            #Y = Y.iloc[:,:out_num_features]
            
            self.batch_id %= self.no
            #print("x.shape=", x.shape)
            #print('X.shape=', X.shape)
            #print(type(x), type(X))
            x = np.append(x, [X], axis = 0)
            
            y = np.append(y, [Y], axis = 0)
            
        return x, y
    
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        return

**WaterDataGenerator 테스트**

In [ ]:
OUT_STEPS = 24*5

In [ ]:
# OUT_STEPS = 24*3
# wdg = WaterDataGenerator(train_df,
#                          batch_size=128,
#                          input_width = 24*7,
#                          label_width = OUT_STEPS,
#                          shift = OUT_STEPS,
#                          out_num_features = out_num_features,
# #                          out_features = out_features
#                         )

In [ ]:
# train_df

In [ ]:
# it = iter(wdg)

In [ ]:
# x,y = next(it)
# x.shape, y.shape

In [ ]:
def make_dataset_water(self, data):
#   print("make_dataset_water")
#   print(out_features)
  dg = WaterDataGenerator(
      data,
      batch_size = 128,
      input_width = self.input_width,
      label_width = self.label_width,
      shift = self.label_width,
      out_features = out_features,
      out_num_features = out_num_features,
  )
  #self.dg = dg
  ds = tf.data.Dataset.from_generator(
      lambda: dg,
      output_types=(tf.float32, tf.float32),
      output_shapes=(
        dg.input_shape,
        dg.label_shape
        #[batch_size, train_generator.dim],
        #[batch_size, train_generator.dim],
      )
  )
  return ds

WindowGenerator.make_dataset = make_dataset_water

In [ ]:
def plot2(self, model=None, plot_col=0, max_subplots=3, plot_out_col=0):
  inputs, labels = self.example
  plt.figure(figsize=(10, 8))
  plot_col_index = self.column_indices[plot_col]
  plot_out_col_index = self.column_indices[plot_out_col]
  max_n = min(max_subplots, len(inputs))
  for n in range(max_n):
    plt.subplot(3, 1, n+1)
    plt.ylabel(f'{plot_col} [normed]', fontproperties=fprop)
    plt.plot(self.input_indices, inputs[n, :, plot_col_index],
             label='Inputs', marker='.', zorder=-10)

    if self.label_columns:
      label_col_index = self.label_columns_indices.get(plot_col, None)
      label_out_col_index = self.label_columns_indices.get(plot_out_col, None)
    else:
      label_col_index = plot_col_index
      label_out_col_index = plot_out_col_index

    if label_col_index is None:
      continue

    plt.scatter(self.label_indices, labels[n, :, label_out_col_index],
                label='Labels', c='#2ca02c')
    if model is not None:
      predictions = model(inputs)
      plt.scatter(self.label_indices, predictions[n, :, label_out_col_index],
                  marker=None, label='Predictions',
                  c='#ff7f0e')

    if n == 0:
      plt.legend()

  plt.xlabel('Time [h]')

WindowGenerator.plot2 = plot2

**1일단위로 변환**

In [ ]:
def hour_to_day_mean(array):
    
#     print(array)
    
    time = 24
    array = tf.reduce_mean(tf.reshape(array, [array.shape[0]//time, time]), 1)
#     print(array)
    
    return array

def plot24(self, model=None, plot_col=0, max_subplots=3, plot_out_col=0):
    
  inputs, labels = self.example
  plt.figure(figsize=(10, 8))
  plot_col_index = self.column_indices[plot_col]
  plot_out_col_index = self.column_indices[plot_out_col]
  max_n = min(max_subplots, len(inputs))

  if self.label_columns:
    label_col_index = self.label_columns_indices.get(plot_col, None)
    label_out_col_index = self.label_columns_indices.get(plot_out_col, None)
  else:
    label_col_index = plot_col_index
    label_out_col_index = plot_out_col_index
    
  for n in range(max_n):
    plt.subplot(3, 1, n+1)
    plt.ylabel(f'{df_all.columns[plot_col]} [normed]', fontproperties=fprop)
    
    input_temp = hour_to_day_mean(inputs[n, :, plot_col_index])
    input_temp = (input_temp * (train_std[plot_col] + 1e-6)) + train_mean[plot_col]
#     input_temp = input_temp * train_std[plot_col] + train_mean[plot_col]
    
    plt.plot(
        hour_to_day_mean(self.input_indices),
        input_temp,
        label='Inputs', marker='.', zorder=-10)

    if label_col_index is None:
      continue
    
    label_temp = hour_to_day_mean(labels[n, :, label_out_col_index])
    label_temp = (label_temp * (train_std[plot_col] + 1e-6)) + train_mean[plot_col]
#     label_temp = label_temp * train_std[plot_col] + train_mean[plot_col]
    
    plt.plot(
        hour_to_day_mean(self.label_indices),
        label_temp,
        label='Labels', marker='.', zorder=-10, c='#2ca02c')
    
    if model is not None:
      predictions = model(inputs)
    
      #predictions = predictions * train_std[plot_col] * train_mean[plot_col]
      predict_temp = hour_to_day_mean(predictions[n,:, label_out_col_index])
      predict_temp = (predict_temp * (train_std[plot_col] + 1e-6)) + train_mean[plot_col]
#       predict_temp = predict_temp * train_std[plot_col] + train_mean[plot_col]
    
      plt.plot(
          hour_to_day_mean(self.label_indices),
          predict_temp,
          label='Predictions', marker='.', zorder=-10, c='#ff7f0e')

    if n == 0:
      plt.legend()

  plt.xlabel('Time [day]')

WindowGenerator.plot24 = plot24

**예측률 계산**

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [ ]:
def compa(self, model=None, plot_col=0, max_subplots=3, plot_out_col=0, windows=None, min_max_normailze=False):
       
    if example is not None :
        inputs, labels = windows
    else :
        inputs, labels = self.example
        
        
    plot_out_col_index = self.column_indices[plot_out_col]
    
    if self.label_columns:
        label_out_col_index = self.label_columns_indices.get(plot_out_col, None)
    else:
        label_out_col_index = plot_out_col_index
    
    if model is None:
        return
    
    print("column : " + df_all.columns[plot_col])
    
    mae = 0
    mse = 0
    rmse = 0
    mape = 0
    pred_arr = []
    label_arr = []
    o = 0
    o1 = 0
    p = 0
    nse_sum1 = 0 
    nse_sum2 = 0
    pbias_sum1 = 0
    pbias_sum2 = 0
    
    
    predictions = model(inputs)
    

    
    predictions = predictions * train_std[plot_col] + train_mean[plot_col]
    labels = labels * train_std[plot_col] + train_mean[plot_col]
    
    
#     print(predictions)
    
#     print(labels)
    o1 = np.mean(labels.numpy())
#     print('o1')
#     print(o1)
    
    if min_max_normailze :
#         predictions = predictions * train_std[plot_col] + train_mean[plot_col]
#         labels = labels * train_std[plot_col] + train_mean[plot_col]
        
        predictions = predictions.numpy()
        labels = labels.numpy()
        
        predictions = (predictions - predictions.min()) / (predictions.max() - predictions.min())
        labels = (labels - labels.min()) / (labels.max() - labels.min())

    for n in range(len(inputs)):
        pred_temp = hour_to_day_mean(predictions[n,:,label_out_col_index]) 
        label_temp = hour_to_day_mean(labels[n,:,label_out_col_index])
        
        o = label_temp[4].numpy()
        p = pred_temp[4].numpy()
        
#         print('pred_temp')
#         print(pred_temp[4])
#         print('+++++++++++++')
#         print(label_temp[4])
        
#         print('p')
#         print(p)
#         print('o')
#         print(o)
        
        temp_m = o-p
#         print('temp')
#         print(temp_m)
#         print(o)

#         print(o-o1)
#         print('22222')
        
        nse_sum1 = nse_sum1 + temp_m**2
        nse_sum2 = nse_sum2 + (o-o1)**2
        
    
            
        pbias_sum1 = pbias_sum1 + temp_m
        pbias_sum2 = pbias_sum2 + o
            
#         pred_arr.append(pred_temp[4])
#         label_arr.append(label_temp[4])
    
    
#     print(nse_sum1)
#     print('111111')
#     print(nse_sum2)

#     print('nse_sum1')
#     print(nse_sum1)
#     print('nse_sum2')
#     print(nse_sum2)
#     print('pbias_sum1')
#     print(pbias_sum1)
#     print('pbias_sum2')
#     print(pbias_sum2)
    
        
    nse = 1 - (nse_sum1 / nse_sum2)
    pbias = (pbias_sum1 / pbias_sum2) * 100
    
    
    
    
    print('NSE')
    print(nse)
    
    print('PBIAS')
    print(pbias)
    
    return nse, np.abs(pbias)
    
        
#     mae = mean_absolute_error(label_arr, pred_arr)
#     mse = mean_squared_error(label_arr, pred_arr)
#     rmse = np.sqrt(mse)
#     mape = np.mean(np.abs((np.array(label_arr) - np.array(pred_arr))/np.array(label_arr)) )*100
    
#     print("mae:")  
#     print(mae)
    
#     print("mse:")  
#     print(mse)
    
#     print("rmse")  
#     print(rmse)
    
#     print("mape")  
#     print(mape)
    
    

WindowGenerator.compa = compa

In [ ]:
# np.sqrt(-0.28892756)

In [ ]:
train_df.shape

In [ ]:
#multi_window.compa(elman_model, plot_col=out_features[0])

In [ ]:
# target_col = '용존산소'
# num_features = df[0].shape[1]
# label_columns_indices = {name: i for i, name in enumerate(df[0])}
# target_col_idx = label_columns_indices[target_col]
# # target_col_idx
# out_features = [target_col_idx]
# # out_features = [3,5,6,7,8]
# out_num_features = len(out_features)
# out_num_features

In [ ]:
def compile_and_fit(model, window, patience=1000):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')

  model.compile(loss=tf.losses.MeanSquaredError(),
                optimizer=tf.optimizers.Adam(),
                metrics=[tf.metrics.MeanAbsoluteError()])
#   model.compile(loss=tf.losses.MeanSquaredError(),
#                 optimizer=tf.optimizers.Adam(),
#                 metrics=[tf.metrics.MeanSquaredError()])
  #model.compile(loss=GAIN.RMSE_loss)

  history = model.fit(window.train, epochs=MAX_EPOCHS,
                      validation_data=window.val,
                      callbacks=[early_stopping])
  return history

In [ ]:
def plot_history(history):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.plot(history.history['loss'], label='loss')
    ax.plot(history.history['mean_absolute_error'], label='mae')
#     ax.plot(history.history['mean_squared_error'], label='mse')
    ax.plot(history.history['val_loss'], label='val_loss')
    ax.plot(history.history['val_mean_absolute_error'], label='val_mae')
#     ax.plot(history.history['val_mean_squared_error'], label='val_mse')
    #plt.legend(history.history.keys(), loc='upper right')
    #ax.legend(loc='upper center')
    ax.legend()
    ax.set_xlabel("epochs")
    ax.set_ylabel("loss")
    plt.show()

In [ ]:
multi_window = WindowGenerator(input_width=24*7,
                               label_width=OUT_STEPS,
                               shift=OUT_STEPS,
                               train_df=train_df,
                               val_df=val_df,
                               test_df=test_df,
                               out_features = out_features,
                               out_num_features = out_num_features,
                               )

In [ ]:
multi_val_performance = {}
multi_performance = {}

**Base Line**

In [ ]:
class MultiStepLastBaseline(tf.keras.Model):
  def call(self, inputs):
    #print(inputs[:, -1:, 0:1])
    #return tf.tile(inputs[:, -1:, :out_num_features], [1, OUT_STEPS, 1])
    return tf.tile(inputs[:, -1:, (out_features[0]):(out_features[0]+1)], [1, OUT_STEPS, 1])
    #return tf.tile(inputs[:, -1:, out_features[0]:(out_features[1]+1)], [1, OUT_STEPS, 1])

last_baseline = MultiStepLastBaseline()
last_baseline.compile(loss=tf.losses.MeanSquaredError(),
                      metrics=[tf.metrics.MeanAbsoluteError()])

multi_val_performance = {}
multi_performance = {}

multi_val_performance['BaseLine'] = last_baseline.evaluate(multi_window.val.repeat(-1), steps=100)
multi_performance['BaseLine'] = last_baseline.evaluate(multi_window.test.repeat(-1), verbose=0, steps=100)
print('val performance =', multi_val_performance['BaseLine'])
print('test performance = ', multi_performance['BaseLine'])

**Linear**

In [ ]:

multi_linear_model = tf.keras.Sequential([
    # Take the last time-step.
    # Shape [batch, time, features] => [batch, 1, features]
    tf.keras.layers.Lambda(lambda x: x[:, -1:, :]),
    # Shape => [batch, 1, out_steps*features]
    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

history = compile_and_fit(multi_linear_model, multi_window)

#IPython.display.clear_output()
multi_val_performance['Linear'] = multi_linear_model.evaluate(multi_window.val.repeat(-1), steps=100)
multi_performance['Linear'] = multi_linear_model.evaluate(multi_window.test.repeat(-1), verbose=0, steps=100)
#multi_window.plot(multi_linear_model, plot_col=0)
print('val performance =', multi_val_performance['Linear'])
print('test performance = ', multi_performance['Linear'])

In [ ]:
multi_performance['Linear'] = multi_linear_model.evaluate(multi_window.test.repeat(-1), verbose=0, steps=100)

**Elman_RNN**

In [ ]:
elman_model = tf.keras.Sequential([
    # Take the last time step.
    # Shape [batch, time, features] => [batch, 1, features]
    tf.keras.layers.SimpleRNN(128, return_sequences=False),

    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),

    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

In [ ]:
elman_model = tf.keras.Sequential([
    # Take the last time step.
    # Shape [batch, time, features] => [batch, 1, features]
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
    tf.keras.layers.SimpleRNN(128, return_sequences=False),

    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),

    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

In [ ]:
history = compile_and_fit(elman_model, multi_window)
multi_val_performance['ELMAN_RNN'] = elman_model.evaluate(multi_window.val.repeat(-1), steps=100)
multi_performance['ELMAN_RNN'] = elman_model.evaluate(multi_window.test.repeat(-1), verbose=1, steps=100)
print('val performance =', multi_val_performance['ELMAN_RNN'])
print('test performance = ', multi_performance['ELMAN_RNN'])

In [ ]:
plot_history(history)

In [ ]:
multi_window.plot2(elman_model, plot_col=out_features[0])

**GRU**

In [ ]:
gru_model = tf.keras.Sequential([
    # Take the last time step.
    # Shape [batch, time, features] => [batch, 1, features]
#     tf.keras.layers.GRU(128, return_sequences=True),
#     tf.keras.layers.GRU(128, return_sequences=True),
#     tf.keras.layers.GRU(128, return_sequences=True),
#     tf.keras.layers.GRU(128, return_sequences=True),
#     tf.keras.layers.GRU(128, return_sequences=True),
#     tf.keras.layers.GRU(128, return_sequences=True),
#     tf.keras.layers.GRU(128, return_sequences=True),
#     tf.keras.layers.GRU(128, return_sequences=True),
    tf.keras.layers.GRU(128, return_sequences=False),
    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

In [ ]:
history = compile_and_fit(gru_model, multi_window)
multi_val_performance['GRU'] = gru_model.evaluate(multi_window.val.repeat(-1), steps=100)
multi_performance['GRU'] = gru_model.evaluate(multi_window.test.repeat(-1), verbose=1, steps=100)
print('val performance =', multi_val_performance['GRU'])
print('test performance = ', multi_performance['GRU'])

In [ ]:
multi_window.plot2(gru_model, plot_col=out_features[0])

**LSTM**

In [ ]:
multi_lstm_model = tf.keras.Sequential([
    # Shape [batch, time, features] => [batch, lstm_units]
    # Adding more `lstm_units` just overfits more quickly.
    #tf.keras.layers.LSTM(32, return_sequences=False),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
    tf.keras.layers.LSTM(128, return_sequences=False),
    # Shape => [batch, out_steps*features]
    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

In [ ]:
history = compile_and_fit(multi_lstm_model, multi_window)

#IPython.display.clear_output()

multi_val_performance['LSTM'] = multi_lstm_model.evaluate(multi_window.val)
multi_performance['LSTM'] = multi_lstm_model.evaluate(multi_window.test, verbose=0)
print('val performance =', multi_val_performance['LSTM'])
print('test performance = ', multi_performance['LSTM'])

**CNN model**

In [ ]:
CONV_WIDTH = 7
CONV_LAYER_NO = 1
multi_conv_model = tf.keras.Sequential([
    # Shape [batch, time, features] => [batch, CONV_WIDTH, features]
    tf.keras.layers.Lambda(lambda x: x[:, -(CONV_WIDTH*CONV_LAYER_NO-CONV_LAYER_NO+1):, :]),
] + [
    # Shape => [batch, 1, conv_units]
    tf.keras.layers.Conv1D(1024, activation='relu', kernel_size=(CONV_WIDTH)) for i in range(CONV_LAYER_NO)
] + [
    # Shape => [batch, 1,  out_steps*features]
    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

In [ ]:
history = compile_and_fit(multi_conv_model, multi_window)

#IPython.display.clear_output()

multi_val_performance['Conv'] = multi_conv_model.evaluate(multi_window.val)
multi_performance['Conv'] = multi_conv_model.evaluate(multi_window.test, verbose=0)
print('val performance =', multi_val_performance['Conv'])
print('test performance = ', multi_performance['Conv'])

In [ ]:
multi_val_performance['Conv'] = multi_conv_model.evaluate(multi_window.val)
multi_performance['Conv'] = multi_conv_model.evaluate(multi_window.test, verbose=0)
print('val performance =', multi_val_performance['Conv'])
print('test performance = ', multi_performance['Conv'])

**performance**

In [ ]:
x = np.arange(len(multi_performance))
width = 0.3


metric_name = 'mean_absolute_error'
metric_index = multi_conv_model.metrics_names.index('mean_absolute_error')
val_mae = [v[metric_index] for v in multi_val_performance.values()]
test_mae = [v[metric_index] for v in multi_performance.values()]

plt.figure()
plt.bar(x - 0.17, val_mae, width, label='Validation')
plt.bar(x + 0.17, test_mae, width, label='Test')
plt.xticks(ticks=x, labels=multi_performance.keys(),
           rotation=45)
plt.ylabel(f'MAE (average over all times and outputs)')
_ = plt.legend()
plt.show()

In [ ]:
val_nse = {}
val_pbias = {}

In [ ]:
val_nse['BaseLine'], val_pbias['BaseLine'] = multi_window.compa(last_baseline, plot_col=out_features[0], windows=multi_window.example3, min_max_normailze=False)

In [ ]:
val_nse['Linear'], val_pbias['Linear'] = multi_window.compa(multi_linear_model, plot_col=out_features[0], windows=multi_window.example3, min_max_normailze=False)

In [ ]:
val_nse['Elman'], val_pbias['Elman'] = multi_window.compa(elman_model, plot_col=out_features[0], windows=multi_window.example3, min_max_normailze=False)

In [ ]:
val_nse['GRU'], val_pbias['GRU'] = multi_window.compa(gru_model, plot_col=out_features[0], windows=multi_window.example3, min_max_normailze=False)

In [ ]:
val_nse['LSTM'], val_pbias['LSTM'] = multi_window.compa(multi_lstm_model, plot_col=out_features[0], windows=multi_window.example3, min_max_normailze=False)

In [ ]:
val_nse['CNN'], val_pbias['CNN'] = multi_window.compa(multi_conv_model, plot_col=out_features[0], windows=multi_window.example3, min_max_normailze=False)

In [ ]:
x = np.arange(len(val_nse))
x
width = 0.35

plt.figure()
plt.bar(x, val_pbias.values(), 0.3, label='PBIAS' )
plt.bar(x + width, val_nse.values(), 0.3, label='NSE')
plt.xticks(x,val_nse.keys(), rotation=45)
_ = plt.legend()
plt.show()

In [ ]:
multi_window.plot24(gru_model, plot_col=out_features[0])